In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
from torchtext.data import Field, BucketIterator

import numpy as np
from torchtext import data

import random
import math
import time

from transformers import T5Model,T5Tokenizer

In [2]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')

In [3]:
tokenizer.vocab_size

32100

In [4]:
tokens = tokenizer.tokenize('Hello world how are you?')

print(tokens)

['▁Hello', '▁world', '▁how', '▁are', '▁you', '?']


In [5]:
indexes = tokenizer.convert_tokens_to_ids(tokens)

print(indexes)

[8774, 296, 149, 33, 25, 58]


In [6]:
init_token = tokenizer.pad_token
eos_token = tokenizer.eos_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)

<pad> </s> <pad> <unk>


In [7]:
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

0 1 0 2


In [8]:
max_input_length = tokenizer.max_model_input_sizes['t5-small']

print(max_input_length)

512


In [9]:
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

In [10]:
SRC = Field(batch_first = True,
          use_vocab = False,
          tokenize = tokenize_and_cut,
          preprocessing = tokenizer.convert_tokens_to_ids,
          init_token = init_token_idx,
          eos_token = eos_token_idx,
          pad_token = pad_token_idx,
          unk_token = unk_token_idx)

TRG = Field(batch_first = True,
          use_vocab = False,
          tokenize = tokenize_and_cut,
          preprocessing = tokenizer.convert_tokens_to_ids,
          init_token = init_token_idx,
          eos_token = eos_token_idx,
          pad_token = pad_token_idx,
          unk_token = unk_token_idx)

/home/gauranga/anaconda3/envs/pt/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [11]:
fields = [('src', SRC), ('trg', TRG)]

In [12]:
train_data = data.TabularDataset.splits(
                path = '',
                train = 'squad.csv',
                format = 'csv',
                fields = fields,
                skip_header = True)

train_data , valid_data = train_data[0].split(split_ratio=0.98,
                                             random_state = random.seed(1234))

/home/gauranga/anaconda3/envs/pt/lib/python3.8/site-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/home/gauranga/anaconda3/envs/pt/lib/python3.8/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


In [13]:
print(len(train_data.examples))
print(len(valid_data.examples))

57226
1168


In [14]:
print(vars(train_data.examples[25000]))

{'src': [2625, 3, 10, 8, 9398, 13, 4353, 7041, 63, 54, 43, 359, 11041, 11, 569, 7702, 5, 778, 18677, 53, 5234, 33, 1086, 5065, 49, 11, 6879, 145, 70, 803, 5, 822, 3, 10, 33, 778, 42, 1480, 18677, 53, 5234, 2389, 5065, 49, 11, 6879, 145, 70, 803, 58], 'trg': [778]}


In [15]:
src_tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[2000])['src'])
trg_tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[2000])['trg'])

print(src_tokens)
print(trg_tokens)

['▁context', '▁', ':', '▁one', '▁way', '▁of', '▁', 'proving', '▁that', '▁', 'a', '▁given', '▁disease', '▁is', '▁"', 'in', 'fect', 'ious', '",', '▁is', '▁to', '▁satisfy', '▁', 'koch', "'", 's', '▁post', 'ulate', 's', '▁(', 'first', '▁proposed', '▁by', '▁', 'rob', 'er', 't', '▁', 'koch', '),', '▁which', '▁demands', '▁that', '▁the', '▁infectious', '▁agent', '▁be', '▁identified', '▁only', '▁in', '▁patients', '▁and', '▁not', '▁in', '▁healthy', '▁controls', ',', '▁and', '▁that', '▁patients', '▁who', '▁contract', '▁the', '▁agent', '▁also', '▁develop', '▁the', '▁disease', '.', '▁question', '▁', ':', '▁what', '▁must', '▁an', '▁infectious', '▁agent', '▁only', '▁be', '▁identified', '▁in', '▁to', '▁satisfy', '▁the', '▁first', '▁of', '▁', 'koch', "'", 's', '▁post', 'ulate', 's', '?']
['▁patients', '▁and', '▁not', '▁in', '▁healthy', '▁controls']


In [16]:
device = torch.device('cuda')

BATCH_SIZE = 24

train_iterator, valid_iterator = BucketIterator.splits(
                                 (train_data, valid_data), 
                                 batch_size = BATCH_SIZE,
                                 device = device,
                                 sort_key=lambda x: len(x.src))

/home/gauranga/anaconda3/envs/pt/lib/python3.8/site-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [17]:
class T5Network(nn.Module):
    def __init__(self):
        
        super().__init__()
        
        self.t5 = T5Model.from_pretrained('t5-small')
        
        self.out = nn.Linear(self.t5.config.to_dict()['d_model'],
                             self.t5.config.to_dict()['vocab_size'])
                
    def forward(self, src, trg):
        
        embedded = self.t5(input_ids=src,decoder_input_ids=trg) 
        
        output = self.out(embedded[0])
        
        return output

In [18]:
model = T5Network().cuda()

Some weights of T5Model were not initialized from the model checkpoint at t5-small and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 76,988,544 trainable parameters


In [20]:
# CONVERT ALL MODEL WEIGHTS AND BIASES TO HALF PRECISION
# MODEL SIZE WILL REDUCE
model = model.half()

In [21]:
model.load_state_dict(torch.load('t5_qa_model.pt'))

<All keys matched successfully>

In [22]:
def translate_sentence2(sentence, eval_model, device, max_len = 50):
    
    eval_model.eval()

    src_indexes = [init_token_idx] + sentence + [eos_token_idx]

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)

    trg_indexes = [init_token_idx]

    for i in range(max_len):

        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)
        
        with torch.no_grad():
            output = eval_model(src_tensor, trg_tensor)
        
        pred_token = output.argmax(2)[:,-1].item()
        
        trg_indexes.append(pred_token)

        if pred_token == eos_token_idx:
            break

    return trg_indexes[1:]

In [23]:
idxs = random.sample(range(0, len(valid_data.examples)), 20)
for i in idxs:
    src = vars(valid_data.examples[i])['src']
    trg = vars(valid_data.examples[i])['trg']
    translation = translate_sentence2(src, model, device)

    print(f"SRC : {''.join(tokenizer.convert_ids_to_tokens(src))}")
    print(f"TRG : {''.join(tokenizer.convert_ids_to_tokens(trg))}")
    print(f"PRED : {''.join(tokenizer.convert_ids_to_tokens(translation))}\n")

SRC : ▁context▁:▁nasser▁played▁a▁significant▁part▁in▁the▁strengthening▁of▁african▁solidarity▁in▁the▁late▁1950s▁and▁early▁1960s,▁although▁his▁continental▁leadership▁role▁had▁increasingly▁passed▁to▁algeria▁since▁1962.▁during▁this▁period,▁nasser▁made▁egypt▁a▁refuge▁for▁anti-colonial▁leaders▁from▁several▁african▁countries▁and▁allowed▁the▁broadcast▁of▁anti-colonial▁propaganda▁from▁cairo.▁beginning▁in▁1958,▁nasser▁had▁a▁key▁role▁in▁the▁discussions▁among▁african▁leaders▁that▁led▁to▁the▁establishment▁of▁the▁organisation▁of▁african▁unity▁(oau)▁in▁1963.▁question▁:▁what▁continent▁did▁nasser▁help▁to▁attain▁political▁stability?
TRG : ▁african
PRED : ▁african</s>

SRC : ▁context▁:▁the▁university▁houses▁the▁following▁public▁broadcasting▁stations:▁kjhk,▁a▁student-run▁campus▁radio▁station,▁kujh-lp,▁an▁independent▁station▁that▁primarily▁broadcasts▁public▁affairs▁programs,▁and▁kanu,▁the▁npr-affiliated▁radio▁station.▁kansas▁public▁radio▁station▁kanu▁was▁one▁of▁the▁first▁public▁radio▁stations▁in▁the▁nation

In [24]:
def return_answer(context,query):
    txt = 'context : ' + CONTEXT.lower() + ' question : ' + QUERY.lower()
    txt_tokens = tokenizer.tokenize(txt)
    txt_ids = tokenizer.convert_tokens_to_ids(txt_tokens)
    pred = translate_sentence2(txt_ids, model, device)
    pred_tokens = tokenizer.convert_ids_to_tokens(pred)
    
    return ''.join(pred_tokens)

In [25]:
CONTEXT = "harry is playing with his dog. the dog is twenty years old."
QUERY = "who is harry playing with ?"

In [26]:
return_answer(CONTEXT,QUERY)

'▁his▁dog</s>'